In [2]:
# Load env variables and create client
import base64
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-sonnet-4-5"

In [3]:
# Helper functions
from anthropic.types import Message


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=1024,
):
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join([block.text for block in message.content if block.type == "text"])

In [5]:
# TODO: Read image data, feed into Claude
with open("pdf/earth.pdf", "rb") as image_file:
    document_data = base64.b64encode(image_file.read()).decode("utf-8")
    
messages = []

article_text = "Earth's atmosphere and oceans were formed by volcanic activity and outgassing. Volcanic eruptions released gases such as water vapor, carbon dioxide, nitrogen, and other trace gases into the atmosphere. As the planet cooled, the water vapor condensed to form oceans. Additionally, comets and asteroids that collided with Earth may have contributed water and other volatiles."

add_user_message(
    messages, 
    [
        {
            "type": "document",
            "source": {
                "type": "text",
                "media_type": "text/plain",
                "data": article_text,
            },
            "title": "earth.pdf",
            "citations": { 
                "enabled": True
            }
        },
        # {
        #     "type": "document",
        #     "source": {
        #         "type": "base64",
        #         "media_type": "application/pdf",
        #         "data": document_data,
        #     },
        #     "title": "earth.pdf",
        #     "citations": { 
        #         "enabled": True
        #     }
        # },
        {
            "type": "text",
            "text": "How were Earth's atmosphere and oceans were formed?"
        }
    ]   
)

chat(messages)

Message(id='msg_01SmjvGnzeHjodBiFBGouRFA', content=[TextBlock(citations=[CitationCharLocation(cited_text="Earth's atmosphere and oceans were formed by volcanic activity and outgassing. ", document_index=0, document_title='earth.pdf', end_char_index=79, file_id=None, start_char_index=0, type='char_location')], text="Earth's atmosphere and oceans were formed by volcanic activity and outgassing.", type='text'), TextBlock(citations=None, text=' ', type='text'), TextBlock(citations=[CitationCharLocation(cited_text='Volcanic eruptions released gases such as water vapor, carbon dioxide, nitrogen, and other trace gases into the atmosphere. ', document_index=0, document_title='earth.pdf', end_char_index=203, file_id=None, start_char_index=79, type='char_location')], text='Volcanic eruptions released gases such as water vapor, carbon dioxide, nitrogen, and other trace gases into the atmosphere.', type='text'), TextBlock(citations=None, text=' ', type='text'), TextBlock(citations=[CitationCharLoc